In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain.chat_models import ChatOpenAI

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
KEY= os.getenv("OPENAI_API_KEY")

In [5]:
print(KEY)

sk-proj-vO80nT6g01gBnISqVh2HT3BlbkFJv05tL0ZoPsvzP6PWw0YJ


In [6]:
llm= ChatOpenAI(openai_api_key = KEY, model_name= "gpt-3.5-turbo",temperature=0.5)

c:\Users\makin\mcqsgen\env\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [7]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [8]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [9]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [10]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [11]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

c:\Users\makin\mcqsgen\env\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [12]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [13]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [14]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [15]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [16]:
data = r"C:\Users\makin\mcqsgen\data.txt"

In [17]:
data

'C:\\Users\\makin\\mcqsgen\\data.txt'

In [18]:
with open(data, 'r') as file:
    TEXT = file.read()

In [19]:
print(TEXT)

"Heart condition" redirects here. For other uses, see Heart condition (disambiguation).
Cardiovascular disease

Micrograph of a heart with fibrosis (yellow) and amyloidosis (brown). Movat's stain.
Specialty	Cardiology
Symptoms	Chest pain, shortness of breath, fatigue, loss of consciousness
Complications	Heart failure, heart attack, stroke, aneurysm, peripheral artery disease, sudden cardiac arrest.[1]
Usual onset	Older adults[2]
Types	Coronary artery diseases, stroke, heart failure, hypertensive heart disease, rheumatic heart disease, cardiomyopathy,[3][4] Arrhythmia
Risk factors	Diabetes,[3] high blood lipids, excess weight, Smoking, excessive drug use, and excessive alcohol intake
Prevention	Healthy eating, exercise, avoiding tobacco smoke, limited alcohol intake,[3] Overall lifestyle changes
Treatment	Treating high blood pressure, high blood lipids, diabetes[3]
Medication	Aspirin, beta blockers, blood thinners
Deaths	17.9 million / 32% (2015)[5]
Cardiovascular disease (CVD) is any d

In [20]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [21]:
NUMBER=5 
SUBJECT="Cardiovascular Disease"
TONE="Easy"

In [22]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

c:\Users\makin\mcqsgen\env\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:"Heart condition" redirects here. For other uses, see Heart condition (disambiguation).
Cardiovascular disease

Micrograph of a heart with fibrosis (yellow) and amyloidosis (brown). Movat's stain.
Specialty	Cardiology
Symptoms	Chest pain, shortness of breath, fatigue, loss of consciousness
Complications	Heart failure, heart attack, stroke, aneurysm, peripheral artery disease, sudden cardiac arrest.[1]
Usual onset	Older adults[2]
Types	Coronary artery diseases, stroke, heart failure, hypertensive heart disease, rheumatic heart disease, cardiomyopathy,[3][4] Arrhythmia
Risk factors	Diabetes,[3] high blood lipids, excess weight, Smoking, excessive drug use, and excessive alcohol intake
Prevention	Healthy eating, exercise, avoiding tobacco smoke, limited alcohol intake,[3] Overall lifestyle changes
Treatment	Treating high blood pressure, high blood lipids, diabetes[3]
Medication	Aspi

In [23]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:18766
Prompt Tokens:17962
Completion Tokens:804
Total Cost:0.028551


In [34]:
response

{'text': '"Heart condition" redirects here. For other uses, see Heart condition (disambiguation).\nCardiovascular disease\n\nMicrograph of a heart with fibrosis (yellow) and amyloidosis (brown). Movat\'s stain.\nSpecialty\tCardiology\nSymptoms\tChest pain, shortness of breath, fatigue, loss of consciousness\nComplications\tHeart failure, heart attack, stroke, aneurysm, peripheral artery disease, sudden cardiac arrest.[1]\nUsual onset\tOlder adults[2]\nTypes\tCoronary artery diseases, stroke, heart failure, hypertensive heart disease, rheumatic heart disease, cardiomyopathy,[3][4] Arrhythmia\nRisk factors\tDiabetes,[3] high blood lipids, excess weight, Smoking, excessive drug use, and excessive alcohol intake\nPrevention\tHealthy eating, exercise, avoiding tobacco smoke, limited alcohol intake,[3] Overall lifestyle changes\nTreatment\tTreating high blood pressure, high blood lipids, diabetes[3]\nMedication\tAspirin, beta blockers, blood thinners\nDeaths\t17.9 million / 32% (2015)[5]\nCa

In [37]:
quiz = response.get("quiz")

In [38]:
json.loads(quiz)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [39]:
print(quiz)

### RESPONSE_JSON
{
    "1": {
        "mcq": "What is the leading cause of death worldwide except in Africa?",
        "options": {
            "a": "Cancer",
            "b": "Cardiovascular disease",
            "c": "Respiratory diseases",
            "d": "Infectious diseases"
        },
        "correct": "b"
    },
    "2": {
        "mcq": "Which dietary pattern has been shown to improve cardiovascular outcomes?",
        "options": {
            "a": "High-sugar diet",
            "b": "Low-carb diet",
            "c": "Mediterranean diet",
            "d": "Fast food diet"
        },
        "correct": "c"
    },
    "3": {
        "mcq": "What is the recommended amount of moderate exercise per week for cardiovascular health?",
        "options": {
            "a": "30 minutes",
            "b": "1 hour",
            "c": "2 hours and 30 minutes",
            "d": "5 hours"
        },
        "correct": "c"
    },
    "4": {
        "mcq": "Which type of medication has been f

In [46]:
import ast
quiz_dict = ast.literal_eval(quiz)
quiz_json = json.dumps(quiz_dict)
parsed_quiz = json.loads(quiz_json)

In [49]:
parsed_quiz

{'1': {'mcq': 'What is the leading cause of death worldwide except in Africa?',
  'options': {'a': 'Cancer',
   'b': 'Cardiovascular disease',
   'c': 'Respiratory diseases',
   'd': 'Infectious diseases'},
  'correct': 'b'},
 '2': {'mcq': 'Which dietary pattern has been shown to improve cardiovascular outcomes?',
  'options': {'a': 'High-sugar diet',
   'b': 'Low-carb diet',
   'c': 'Mediterranean diet',
   'd': 'Fast food diet'},
  'correct': 'c'},
 '3': {'mcq': 'What is the recommended amount of moderate exercise per week for cardiovascular health?',
  'options': {'a': '30 minutes',
   'b': '1 hour',
   'c': '2 hours and 30 minutes',
   'd': '5 hours'},
  'correct': 'c'},
 '4': {'mcq': 'Which type of medication has been found to be effective in preventing further cardiovascular disease in people with a history of cardiovascular disease?',
  'options': {'a': 'Antibiotics',
   'b': 'Statins',
   'c': 'Vitamin C',
   'd': 'Niacin'},
  'correct': 'b'},
 '5': {'mcq': 'What is the main fo

In [48]:
quiz_dict

{'1': {'mcq': 'What is the leading cause of death worldwide except in Africa?',
  'options': {'a': 'Cancer',
   'b': 'Cardiovascular disease',
   'c': 'Respiratory diseases',
   'd': 'Infectious diseases'},
  'correct': 'b'},
 '2': {'mcq': 'Which dietary pattern has been shown to improve cardiovascular outcomes?',
  'options': {'a': 'High-sugar diet',
   'b': 'Low-carb diet',
   'c': 'Mediterranean diet',
   'd': 'Fast food diet'},
  'correct': 'c'},
 '3': {'mcq': 'What is the recommended amount of moderate exercise per week for cardiovascular health?',
  'options': {'a': '30 minutes',
   'b': '1 hour',
   'c': '2 hours and 30 minutes',
   'd': '5 hours'},
  'correct': 'c'},
 '4': {'mcq': 'Which type of medication has been found to be effective in preventing further cardiovascular disease in people with a history of cardiovascular disease?',
  'options': {'a': 'Antibiotics',
   'b': 'Statins',
   'c': 'Vitamin C',
   'd': 'Niacin'},
  'correct': 'b'},
 '5': {'mcq': 'What is the main fo

In [50]:
quiz_table_data = []
for key, value in quiz_dict.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [51]:
quiz_table_data

[{'MCQ': 'What is the leading cause of death worldwide except in Africa?',
  'Choices': 'a: Cancer | b: Cardiovascular disease | c: Respiratory diseases | d: Infectious diseases',
  'Correct': 'b'},
 {'MCQ': 'Which dietary pattern has been shown to improve cardiovascular outcomes?',
  'Choices': 'a: High-sugar diet | b: Low-carb diet | c: Mediterranean diet | d: Fast food diet',
  'Correct': 'c'},
 {'MCQ': 'What is the recommended amount of moderate exercise per week for cardiovascular health?',
  'Choices': 'a: 30 minutes | b: 1 hour | c: 2 hours and 30 minutes | d: 5 hours',
  'Correct': 'c'},
 {'MCQ': 'Which type of medication has been found to be effective in preventing further cardiovascular disease in people with a history of cardiovascular disease?',
  'Choices': 'a: Antibiotics | b: Statins | c: Vitamin C | d: Niacin',
  'Correct': 'b'},
 {'MCQ': 'What is the main focus of initial treatment for cardiovascular disease?',
  'Choices': 'a: Surgery | b: Diet and lifestyle intervent

In [52]:
quiz=pd.DataFrame(quiz_table_data)

In [54]:
quiz

,MCQ,Choices,Correct
0,What is the leading cause of death worldwide e...,a: Cancer | b: Cardiovascular disease | c: Res...,b
1,Which dietary pattern has been shown to improv...,a: High-sugar diet | b: Low-carb diet | c: Med...,c
2,What is the recommended amount of moderate exe...,a: 30 minutes | b: 1 hour | c: 2 hours and 30 ...,c
3,Which type of medication has been found to be ...,a: Antibiotics | b: Statins | c: Vitamin C | d...,b
4,What is the main focus of initial treatment fo...,a: Surgery | b: Diet and lifestyle interventio...,b


In [53]:
quiz.to_csv("Cardovascular_Disease",index=False)

In [55]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'07_09_2024_03_59_05'